### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# BERT

*Author: HuggingFace Team*

**Bidirectional Encoder Representations from Transformers.**

_ | _
- | -
![alt](https://pytorch.org/assets/images/bert1.png) | ![alt](https://pytorch.org/assets/images/bert2.png)


### Model Description

BERT was released together with the paper [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) by Jacob Devlin et al. The model is based on the Transformer architecture introduced in [Attention Is All You Need](https://arxiv.org/abs/1706.03762) by Ashish Vaswani et al and has led to significant improvements on a wide range of downstream tasks.

Here are 8 models based on BERT with [Google's pre-trained models](https://github.com/google-research/bert) along with the associated Tokenizer.
It includes:
- `bertTokenizer`: perform end-to-end tokenization, i.e. basic tokenization followed by WordPiece tokenization
- `bertModel`: raw BERT Transformer model (fully pre-trained)
- `bertForMaskedLM`: BERT Transformer with the pre-trained masked language modeling head on top (fully pre-trained)
- `bertForNextSentencePrediction`: BERT Transformer with the pre-trained next sentence prediction classifier on top (fully pre-trained)
- `bertForPreTraining`: BERT Transformer with masked language modeling head and next sentence prediction classifier on top (fully pre-trained)
- `bertForSequenceClassification`: BERT Transformer with a sequence classification head on top (BERT Transformer is pre-trained, the sequence classification head is only initialized and has to be trained)
- `bertForMultipleChoice`: BERT Transformer with a multiple choice head on top (used for task like Swag) (BERT Transformer is pre-trained, the multiple choice classification head is only initialized and has to be trained)
- `bertForTokenClassification`: BERT Transformer with a token classification head on top (BERT Transformer is pre-trained, the token classification head is only initialized and has to be trained)
- `bertForQuestionAnswering`: BERT Transformer with a token classification head on top (BERT Transformer is pre-trained, the token classification head is only initialized and has to be trained)

### Requirements

Unlike most other PyTorch Hub models, BERT requires a few additional Python packages to be installed.

In [1]:
%%bash

pip install tqdm boto3 requests regex

  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
import os
import re

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df


In [3]:
train, test = download_and_load_datasets()

84131840/84125825 [==============================] - 3s 0us/step


In [4]:
train.head()


,sentence,sentiment,polarity
0,I've been watching this movie by hoping to fin...,3,0
1,Carlo Verdone once managed to combine superb c...,4,0
2,One of the more satisfying Western all'italian...,7,1
3,I love this movie. My only disappointment was ...,10,1
4,The acting was horrendous as well as the scree...,3,0


In [72]:
[i for i in train['sentence'][0:10]]

['I\'ve been watching this movie by hoping to find a pretty and interesting story yet the story line wasn\'t good at all. The play of the actors weren\'t any better.<br /><br />Of course Shahrukh Khan was there yet he wasn\'t enough to make this movie "credible" and interesting.<br /><br />I\'ve read that this movie was based on the novel of Flaubert "Madame Bovary" yet for me I didn\'t see it matching with the Indian mentality.<br /><br />In general we buy movie to dream and have a good time, not to waste our time and change our mood into worse. I just can\'t understand how it could get such a "high" vote with an average of 6.8/10.<br /><br />So it\'s the kind of movie you should run away & ignore because there is nothing to appreciate in it! You will just waste your time unless if you like "dark movie" with "strange and non sense story".',
 'Carlo Verdone once managed to combine superb comedy with smart and subtle social analysis and criticism.<br /><br />Then something happened, and

In [0]:
trainloader = torch.utils.data.DataLoader(train, batch_size=4,
                                          shuffle=True, num_workers=2)

In [54]:
bert.run_classifer.InputExample

NameError: ignored

### Example

Here is an example on how to tokenize the input text with `bertTokenizer`, and then get the hidden states computed by `bertModel` or predict masked tokens using `bertForMaskedLM`. The example also includes snippets showcasing how to use `bertForNextSentencePrediction`, `bertForQuestionAnswering`, `bertForSequenceClassification`, `bertForMultipleChoice`, `bertForTokenClassification`, and `bertForPreTraining`.

In [5]:
### First, tokenize the input
import torch
tokenizer = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'bertTokenizer', 'bert-base-cased', do_basic_tokenize=False)

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

Downloading: "https://github.com/huggingface/pytorch-pretrained-BERT/archive/master.zip" to /root/.cache/torch/hub/master.zip
W0723 16:04:03.172120 140287905322880 tokenization_bert.py:190] The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 213450/213450 [00:00<00:00, 2378126.32B/s]


In [6]:
print(text)
print(tokenized_text)
print(indexed_tokens)

[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]
['[CLS]', 'Who', 'was', 'Jim', 'He', '##nson', '?', '[SEP]', 'Jim', 'He', '##nson', 'was', 'a', 'puppet', '##eer', '[SEP]']
[101, 2627, 1108, 3104, 1124, 15703, 136, 102, 3104, 1124, 15703, 1108, 170, 16797, 8284, 102]


In [0]:
### Get the hidden states computed by `bertModel`
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])


In [13]:
print(tokenizer.encode("Hello, my dog is cute"))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Hello, my dog is cute")))

[8667, 28136, 1139, 3676, 1110, 10509]
[8667, 28136, 1139, 3676, 1110, 10509]


In [33]:
model = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'bertForSequenceClassification', 'bert-base-cased', num_labels=2)

input_id = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0) #batch_size 1
label = torch.tensor(1).unsqueeze(0) # batch_size 1
  
outputs = model(input_id, labels = label)

loss, logit = outputs

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-pretrained-BERT_master


In [39]:
torch.nn.functional.softmax(logit, dim=1)

tensor([[0.3678, 0.6322]], grad_fn=<SoftmaxBackward>)

In [0]:
lr = 1e-3
num_total_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1

### Previously BertAdam optimizer was instantiated like this:
optimizer = BertAdam(model.parameters(), lr=lr, schedule='warmup_linear', warmup=warmup_proportion, t_total=num_total_steps)
### and used like this:

for batch in train_data:
    loss = model(batch)
    loss.backward()
    optimizer.step()

In [14]:

model = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'bertForSequenceClassification', 'bert-base-cased', num_labels=2)
model.eval()

# Predict the sequence classification logits
with torch.no_grad():
    seq_classif_logits = model(tokens_tensor, segments_tensors)

# Or get the sequence classification loss (set model to train mode before if used for training)
labels = torch.tensor([1])
seq_classif_loss = model(tokens_tensor, segments_tensors, labels=labels)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-pretrained-BERT_master
